# Modlling for engagement 

now we have the features and the lables, we are ready for modelling 
- Classification vs regression
- 9 vs 5 vs 3 scales lables 
- 2.5 vs 5 s window (as is features, average featuers and concatenate features)
- Two-stream Fusion on RGB + Flow 

This should be done over:
- different network artchictures
- cross different familys cross-validation 

Later on:
- Handcrafted features 

### imports and vars

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import itertools
import pickle
import warnings
warnings.filterwarnings('ignore')

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import logging, sys
logging.disable(sys.maxsize)

import smote_variants as sv
from collections import Counter
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from operator import itemgetter

lables_path = './labels/'
features_path = './features/'
results_path = './modelling_results/'

### Keras modeling class 

### help functions 


In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker
from hpbandster.optimizers import BOHB as BOHB
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

import tensorflow_addons as tfa

import logging
logging.basicConfig(level=logging.ERROR)

class KerasWorker(Worker):
    def __init__(self, input_shape, output_shape, problemType,
                 x_train, y_train, x_validation, y_validation,
                 x_test, y_test, shared_directory, **kwargs):
            super().__init__(**kwargs)
            self.input_shape = (input_shape, )
            self.num_classes = output_shape
            self.batch_size = 64
            self.save_dic = shared_directory
            
            self.problemType = problemType

            self.x_train, self.y_train = x_train, y_train
            self.x_validation, self.y_validation = x_validation, y_validation
            self.x_test, self.y_test = x_test, y_test

    def compute(self, config, budget, working_directory, *args, **kwargs):
            model = Sequential()
            model.add(Dense(units=config['start_neurons_units'],
                            # activation=config['start_neurons_activation'],
                            activation='relu',
                            input_shape=self.input_shape))


            if config['num_dense_layers'] > 1:
                model.add(Dense(units=config['dense1_units'],
                                # activation=config['dense1_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout1_rate']))

            if config['num_dense_layers'] > 2:
                model.add(Dense(units=config['dense2_units'],
                                # activation=config['dense2_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout2_rate']))

            model.add(Dense(self.num_classes, activation='softmax'))


            if config['optimizer'] == 'Adam':
                    optimizer = tf.keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = tf.keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])
            
            loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
            METRICS = [
                  tf.keras.metrics.TruePositives(name='tp'),
                  tf.keras.metrics.FalsePositives(name='fp'),
                  tf.keras.metrics.TrueNegatives(name='tn'),
                  tf.keras.metrics.FalseNegatives(name='fn'),
                  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  tf.keras.metrics.AUC(name='auc'),
                  tf.keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
                  tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes)
            ]

            #METRICS.append(tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes))
            
            the_columns = ['thisloss','tp','fp','tn','fn','acc','prec','rec','auc','BC','MCC']
            
            model.compile(
                loss=loss_fn,
                optimizer=optimizer,
                metrics=METRICS
            )

            # model.summary()
            _history = model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=int(budget),
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            val_acc_per_epoch = _history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

            model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=best_epoch,
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
            val_score = model.evaluate(self.x_validation, self.y_validation, verbose=0)
            test_score = model.evaluate(self.x_test, self.y_test, verbose=0)
            
            #print(test_score)

            resultsDF = pd.DataFrame([train_score,val_score,test_score],
                                     columns=the_columns,
                                     index=["train_score", "val_score", "test_score"],)
            # print(resultsDF)
            test_predictions_baseline = model.predict(self.x_test)
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_true.out'), self.y_test, delimiter=',')
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_pred.out'), test_predictions_baseline, delimiter=',')

            return ({
                'loss': test_score[0],  
                'info':  resultsDF.to_dict('index')
            })
        
#             #import IPython; IPython.embed()
#             return ({
#                 'loss': 1-val_score[1], # remember: HpBandSter always minimizes!
#                 'info': {'test accuracy': test_score[1],
#                             'train accuracy': train_score[1],
#                             'validation accuracy': val_score[1],
#                             'number of parameters': model.count_params(),
#                         }

#             })


    @staticmethod
    def get_configspace():
            """
            It builds the configuration space with the needed hyperparameters.
            It is easily possible to implement different types of hyperparameters.
            Beside float-hyperparameters on a log scale, it is also able to handle categorical input parameter.
            :return: ConfigurationsSpace-Object
            """
            cs = CS.ConfigurationSpace()

            lr = CSH.UniformFloatHyperparameter('lr', lower=1e-6, upper=1e-1, default_value='1e-2', log=True)

            # For demonstration purposes, we add different optimizers as categorical hyperparameters.
            # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
            # SGD has a different parameter 'momentum'.
            optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

            sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

            cs.add_hyperparameters([lr, optimizer, sgd_momentum])



            num_dense_layers =  CSH.UniformIntegerHyperparameter('num_dense_layers', lower=1, upper=3, default_value=2)

            start_neurons_units = CSH.UniformIntegerHyperparameter('start_neurons_units', lower=32, upper=512, default_value=32, log=True)
            dense1_units = CSH.UniformIntegerHyperparameter('dense1_units', lower=8, upper=128, default_value=16, log=True)
            dense2_units = CSH.UniformIntegerHyperparameter('dense2_units', lower=4, upper=64, default_value=8, log=True)

            cs.add_hyperparameters([num_dense_layers, start_neurons_units, dense1_units, dense2_units])

            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu', 'tanh', 'sigmoid'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu', 'tanh', 'sigmoid'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu', 'tanh', 'sigmoid'])
            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu'])
            #
            # cs.add_hyperparameters([start_neurons_activation, dense1_activation, dense2_activation])

            dropout1_rate = CSH.UniformFloatHyperparameter('dropout1_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
            dropout2_rate = CSH.UniformFloatHyperparameter('dropout2_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)

            cs.add_hyperparameters([dropout1_rate, dropout2_rate])


            # The hyperparameter sgd_momentum will be used,if the configuration
            # contains 'SGD' as optimizer.
            cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
            cs.add_condition(cond)

            # You can also use inequality conditions:
            cond = CS.GreaterThanCondition(dense1_units, num_dense_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(dense2_units, num_dense_layers, 2)
            cs.add_condition(cond)

            return cs


In [3]:
def load_sub_dataset(familiesSet, label_folder, feature_folder, featureType):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])

    for this_family in familiesSet:
        # F10_Interaction_1_P27_rgb.npy
        onlyfiles = [f for f in os.listdir(feature_folder) if
                       os.path.isfile(os.path.join(feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            currData = np.load(os.path.join(feature_folder,this_file))            
            substr = '_'.join(this_file.split('_')[4:])
            this_lable_file = this_file.replace('_'+substr,'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            cutoff = min(len(currData), len(currLabel))
            currData = currData[:cutoff,:]
            currLabel =currLabel[:cutoff]
            
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def fix_data(X, y, shuff=False):
    c = Counter(y)
    print('Before:',X.shape, c)
    min_key, min_count = min(c.items(), key=itemgetter(1))
    print(min_key, min_count)
    if min_count<10:
        indx = np.where(y == min_key)
        dub = X[indx]
#         print(dub.shape)
        X = np.vstack((X, np.tile(dub, (10, 1))))
        y = np.hstack((y, np.repeat(min_key,len(dub)*10)))
        print('Dublicated :',X.shape, Counter(y))
    
        #     sm = resample(n_neighbors=max(min_count-1,1))
    sm = sv.MulticlassOversampling(resample())
    if shuff:
        X, y = shuffle(X, y)
        
    #     X, trainy = sm.fit_resample(X, y)
    X, y = sm.sample(X, y)
    print('After:',X.shape, Counter(y))
    
    return X, y

def load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, label_folder, feature_folder,\
                              prblemType, featureType, num_classes):
    scaler = MinMaxScaler()
    
    #sampeling method: SMOTE, DeepSMOTE, DeepFake?
    # load all train
    trainX, trainy = load_sub_dataset(trainSubjs, label_folder, feature_folder, featureType)
    trainX = scaler.fit_transform(trainX)
    print('Train Data:')
    trainX, trainy = fix_data(trainX, trainy, shuff=True)


    # load validation
    valX, valy = load_sub_dataset(valSubjs, label_folder, feature_folder, featureType)
    valX = scaler.transform(valX)
    print('Val Data:')
    valX, valy = fix_data(valX, valy, shuff=True)
    
    # load all test
    testX, testy = load_sub_dataset(testSubjs, label_folder, feature_folder, featureType)
    testX = scaler.transform(testX)
    print('Test Data:')
    testX, testy = fix_data(testX, testy, shuff=False)

    # one hot encode y
    trainy = tf.keras.utils.to_categorical(trainy,  num_classes=num_classes)
    valy = tf.keras.utils.to_categorical(valy,  num_classes=num_classes)
    testy = tf.keras.utils.to_categorical(testy,  num_classes=num_classes)
    
    return trainX, trainy, valX, valy , testX, testy, num_classes
    
def create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, num_classes):    
    # repeat experiment
    temp = {}
    all_trainSubjs = [
        ['F' + str(i) for i in [1, 2, 3, 4, 5, 6, 8]],
        ['F' + str(i) for i in [1, 2, 3, 4, 5, 6, 7]],
        ['F' + str(i) for i in [1, 2, 3, 5, 17, 8]],
        ['F' + str(i) for i in [4, 7, 10, 11, 13, 17]],
        ['F' + str(i) for i in [6, 8, 10, 11, 13, 17]],
    ]
    all_valSubjs = [
        ['F' + str(i) for i in [11, 17]],
        ['F' + str(i) for i in [8, 10]],
        ['F' + str(i) for i in [10, 13, 6]],
        ['F' + str(i) for i in [1, 2, 3]],
        ['F' + str(i) for i in [4, 7, 5]],
    ]
    all_testSubjs = [
        ['F' + str(i) for i in [7, 10, 13]],
        ['F' + str(i) for i in [11, 13, 17]],
        ['F' + str(i) for i in [7, 4, 11]],
        ['F' + str(i) for i in [5, 6, 8]],
        ['F' + str(i) for i in [1, 2, 3]],
    ]
    
    min_budget = 9
    max_budget = 243
    n_iterations = 50
    num_workers = 12

    for r in range(len(all_trainSubjs)):
        shared_directory = result_folder + '_'+ str(r) 
        if os.path.exists(shared_directory):
            print(shared_directory,' already processed')
            continue
        
        print(shared_directory,' under processing')
        classType = os.path.basename(shared_directory)

        # load data
        trainSubjs = all_trainSubjs[r]
        valSubjs = all_valSubjs[r]
        testSubjs = all_testSubjs[r]
        
        trainX, trainy, valX, valy, testX, testy, num_classes = \
            load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, \
                                      label_folder, feature_folder, prblemType, featureType, num_classes)
        
        
        n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], num_classes        
        
        host = hpns.nic_name_to_host('lo')
        result_logger = hpres.json_result_logger(directory=shared_directory, overwrite=True)
        NS = hpns.NameServer(run_id=classType, host=host, port=0, working_directory=shared_directory)
        ns_host, ns_port = NS.start()
    
        workers = []
        for i in range(num_workers):
            worker = KerasWorker(n_features, n_outputs, prblemType, \
                                 trainX, trainy, valX, valy, testX, testy, \
                                 shared_directory,
                                 run_id=classType,host=host, nameserver=ns_host, nameserver_port=ns_port,
                                 id=i)
            worker.run(background=True)
            workers.append(worker)

        bohb = BOHB(configspace=worker.get_configspace(),
                  run_id=classType,
                  host=host,
                  nameserver=ns_host,
                  nameserver_port=ns_port,
                  result_logger=result_logger,
                  min_budget=min_budget, max_budget=max_budget
                    )
        res = bohb.run(n_iterations=1,  min_n_workers=num_workers)

        id2config = res.get_id2config_mapping()
        incumbent = res.get_incumbent_id()

        print('Best found configuration:', id2config[incumbent]['config'])
        # print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
        # print('A total of %i runs where executed.' % len(res.get_all_runs()))
        # print('Total budget corresponds to %.1f full function evaluations.' % (
        #             sum([r.budget for r in res.get_all_runs()]) / max_budget))

        # store results
        with open(os.path.join(shared_directory, 'results.pkl'), 'wb') as fh:
            pickle.dump(res, fh)

        # shutdown
        bohb.shutdown(shutdown_workers=True)
        NS.shutdown()


### modeling handcrafted 

In [4]:
prblemTypes = ['classification']
featureTypes = ['handcrafted_extra']
classes = [2, 3, 5]
fusionTypes = ['2.5','5','5s']
sampling_methods = ['polynom_fit_SMOTE']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'

    
    print('Working on: ',prblemType, featureType, eng_lvls, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(fusionType)]))
    feature_folder = os.path.join(features_path,'_'.join([featureType,'features',fusionType]))
    
    for sampling_method in sampling_methods:

        resample = getattr(__import__("smote_variants", globals(), locals(), [sampling_method], 0), sampling_method)

        #save results like: rgb_classification_9_2.5_none
        result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),fusionType,sampling_method]))

        create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)


9
Working on:  classification handcrafted_extra 2 2.5
./modelling_results/handcrafted_extra_classification_2_2.5_polynom_fit_SMOTE_0  under processing
Train Data:
Before: (18236, 1860) Counter({1.0: 14902, 0.0: 3334})
0.0 3334
After: (28238, 1860) Counter({1.0: 14902, 0.0: 13336})
Val Data:
Before: (3467, 1860) Counter({1.0: 3003, 0.0: 464})
0.0 464
After: (5787, 1860) Counter({1.0: 3003, 0.0: 2784})
Test Data:
Before: (3916, 1860) Counter({1.0: 3568, 0.0: 348})
0.0 348
After: (7048, 1860) Counter({1.0: 3568, 0.0: 3480})


2021-11-09 04:03:40.353697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-09 04:03:40.358891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-09 04:03:40.359645: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-09 04:03:40.359668: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.48167471074011653, 'dropout2_rate': 0.23768978196725596, 'lr': 3.916614360968384e-05, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 156, 'dense1_units': 39}
./modelling_results/handcrafted_extra_classification_2_2.5_polynom_fit_SMOTE_1  under processing
Train Data:
Before: (17266, 1860) Counter({1.0: 13898, 0.0: 3368})
0.0 3368
After: (27370, 1860) Counter({1.0: 13898, 0.0: 13472})
Val Data:
Before: (3892, 1860) Counter({1.0: 3620, 0.0: 272})
0.0 272
After: (7156, 1860) Counter({1.0: 3620, 0.0: 3536})
Test Data:
Before: (4461, 1860) Counter({1.0: 3955, 0.0: 506})
0.0 506
After: (8003, 1860) Counter({0.0: 4048, 1.0: 3955})
Best found configuration: {'dropout1_rate': 0.8902951949717673, 'dropout2_rate': 0.6862640676015757, 'lr': 2.244440300207414e-05, 'num_dense_layers': 1, 'optimizer': 'Adam', 'start_neurons_units': 370}
./modelling_results/handcrafted_extra_classification_2_2.5_polynom_fit_SMOTE_2  under processing
Trai

After: (5849, 1860) Counter({1.0: 3029, 0.0: 2820})
Best found configuration: {'dropout1_rate': 0.1112214425695892, 'dropout2_rate': 0.3185733158447022, 'lr': 9.088999535688267e-06, 'num_dense_layers': 1, 'optimizer': 'Adam', 'start_neurons_units': 130}
./modelling_results/handcrafted_extra_classification_2_5s_polynom_fit_SMOTE_3  under processing
Train Data:
Before: (5154, 1860) Counter({1.0: 4381, 0.0: 773})
0.0 773
After: (9019, 1860) Counter({0.0: 4638, 1.0: 4381})
Val Data:
Before: (2646, 1860) Counter({1.0: 2140, 0.0: 506})
0.0 506
After: (4164, 1860) Counter({1.0: 2140, 0.0: 2024})
Test Data:
Before: (3152, 1860) Counter({1.0: 2662, 0.0: 490})
0.0 490
After: (5112, 1860) Counter({1.0: 2662, 0.0: 2450})
Best found configuration: {'dropout1_rate': 0.6324934098367755, 'dropout2_rate': 0.2623963443072789, 'lr': 1.5076750077396623e-06, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 82, 'dense1_units': 58}
./modelling_results/handcrafted_extra_classification_2_5s_p

./modelling_results/handcrafted_extra_classification_3_5_polynom_fit_SMOTE_3  under processing
Train Data:
Before: (6342, 1860) Counter({1.0: 5325, 0.0: 936, -1.0: 81})
-1.0 81
After: (16449, 1860) Counter({0.0: 5616, -1.0: 5508, 1.0: 5325})
Val Data:
Before: (3098, 1860) Counter({1.0: 2499, 0.0: 589, -1.0: 10})
-1.0 10
After: (7285, 1860) Counter({1.0: 2499, -1.0: 2430, 0.0: 2356})
Test Data:
Before: (3720, 1860) Counter({1.0: 3130, 0.0: 453, -1.0: 137})
-1.0 137
After: (9452, 1860) Counter({0.0: 3171, -1.0: 3151, 1.0: 3130})
Best found configuration: {'dropout1_rate': 0.619839210891464, 'dropout2_rate': 0.2978140005454872, 'lr': 0.009763006595664496, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 338, 'dense1_units': 8}
./modelling_results/handcrafted_extra_classification_3_5_polynom_fit_SMOTE_4  under processing
Train Data:
Before: (5605, 1860) Counter({1.0: 4920, 0.0: 606, -1.0: 79})
-1.0 79
After: (14666, 1860) Counter({1.0: 4920, -1.0: 4898, 0.0: 4848})
Val Da

Best found configuration: {'dropout1_rate': 0.7844070067357705, 'dropout2_rate': 0.43174649725131947, 'lr': 0.0606936620585775, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 96, 'dense1_units': 9, 'sgd_momentum': 0.8364987574406084}
./modelling_results/handcrafted_extra_classification_5_2.5_polynom_fit_SMOTE_3  under processing
Train Data:
Before: (12707, 1860) Counter({1.0: 8368, 2.0: 2299, 0.0: 1885, -1.0: 155})
-1.0 155
After: (35979, 1860) Counter({0.0: 9425, 2.0: 9196, -1.0: 8990, 1.0: 8368})
Val Data:
Before: (6207, 1860) Counter({1.0: 3553, 2.0: 1473, 0.0: 1157, -1.0: 24})
-1.0 24
After: (13306, 1860) Counter({1.0: 3553, 0.0: 3471, -1.0: 3336, 2.0: 2946})
Test Data:
Before: (7455, 1860) Counter({1.0: 5031, 2.0: 1265, 0.0: 878, -1.0: 279, -2.0: 2})
-2.0 2
Dublicated : (7475, 1860) Counter({1.0: 5031, 2.0: 1265, 0.0: 878, -1.0: 279, -2.0: 22})
After: (25485, 1860) Counter({0.0: 5268, -2.0: 5104, 2.0: 5060, 1.0: 5031, -1.0: 5022})
Best found configuration: {'dro

./modelling_results/handcrafted_extra_classification_5_5s_polynom_fit_SMOTE_2  under processing
Train Data:
Before: (5444, 1860) Counter({1.0: 3256, 2.0: 1226, 0.0: 819, -1.0: 143})
-1.0 143
After: (13642, 1860) Counter({2.0: 3678, -1.0: 3432, 0.0: 3276, 1.0: 3256})
Val Data:
Before: (1915, 1860) Counter({1.0: 1377, 2.0: 295, 0.0: 214, -1.0: 28, -2.0: 1})
-2.0 1
Dublicated : (1925, 1860) Counter({1.0: 1377, 2.0: 295, 0.0: 214, -1.0: 28, -2.0: 11})
After: (7258, 1860) Counter({0.0: 1498, 2.0: 1475, -1.0: 1456, -2.0: 1452, 1.0: 1377})
Test Data:
Before: (3593, 1860) Counter({1.0: 2425, 2.0: 604, 0.0: 541, -1.0: 23})
-1.0 23
After: (9351, 1860) Counter({1.0: 2425, 2.0: 2416, -1.0: 2346, 0.0: 2164})
Best found configuration: {'dropout1_rate': 0.6873188776700668, 'dropout2_rate': 0.5028985017444307, 'lr': 0.003352171583378529, 'num_dense_layers': 3, 'optimizer': 'SGD', 'start_neurons_units': 144, 'dense1_units': 8, 'dense2_units': 38, 'sgd_momentum': 0.9838638959601249}
./modelling_results/